In [ ]:
import requests
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
from datetime import datetime, timedelta
import os

nltk.download('vader_lexicon', quiet=True)

sia = SentimentIntensityAnalyzer()

API_KEY = 'f436f3063b6a49ddb099ab1ded54b6a0'

# Diccionario completo de nombres de empresas de energía con su ticker
energy_companies_dict = {
    'ypf': 'YPF',  # Argentina - Petróleo y gas
    'pampa_energia': 'PAM',  # Argentina - Energía integrada
    'ecopetrol': 'EC',  # Colombia - Petróleo y gas
    'petrobras': 'PBR',  # Brasil - Petróleo y gas
    'interconexion_electrica': 'IESFY',  # Colombia - Transmisión de energía
    'eletrobras': 'EBR',  # Brasil - Generación y transmisión de energía
    'engie_brasil_energia': 'EGIE3.SA',  # Brasil - Generación y distribución de energía
    'vista_energy': 'VIST',  # Argentina - Petróleo y gas
    'aes_andes': 'AES',  # Chile - Generación de energía
    'cemig': 'CIG',  # Brasil - Generación y distribución de energía
    'copel': 'ELP',  # Brasil - Generación y distribución de energía
    'enel_chile': 'ENIC',  # Chile - Generación y distribución de energía
    'cosan': 'CSAN',  # Brasil - Energía y logística
    'ultrapar': 'UGP',  # Brasil - Distribución de combustibles
    'companhia_siderurgica_nacional': 'SID',  # Brasil - Acero, minería y energía
    'vale': 'VALE',  # Brasil - Minería y energía
    'braskem': 'BAK',  # Brasil - Petroquímica
    'centrais_eletricas_brasileiras': 'EBR',  # Brasil - Generación y transmisión de energía
    'companhia_paranaense_de_energia': 'ELP',  # Brasil - Generación y distribución de energía
    'light': 'LGSXY',  # Brasil - Distribución de energía
    'transmissora_alianca_de_energia_eletrica': 'TAEE11.SA',  # Brasil - Transmisión de energía
    'alupar_investimento': 'ALUP11.SA',  # Brasil - Transmisión de energía
    'eneva': 'ENEV3.SA',  # Brasil - Generación de energía
    'raizen': 'RAIZ4.SA',  # Brasil - Biocombustibles y distribución
    'petrorio': 'PRIO3.SA',  # Brasil - Petróleo y gas
    '3r_petroleum': 'RRRP3.SA',  # Brasil - Petróleo y gas
    'enauta_participacoes': 'ENAT3.SA',  # Brasil - Petróleo y gas
    'pinfra': 'PINFRA.MX',  # México - Infraestructura y energía
    'geb': 'GEB.CL',  # Colombia - Energía y gas natural
}

# Definir rutas relativas
WEBSCRAPING_PATH = os.path.join('..', 'datasets', 'dataset_webscraping')
SENTIMENT_PATH = os.path.join('..', 'datasets', 'dataset_sentimiento')

def get_news(company_name, days=30, max_results=100):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    query = f'"{company_name}" OR "{energy_companies_dict[company_name]}"'
    
    url = f'https://newsapi.org/v2/everything?q={query}&from={start_date.date()}&to={end_date.date()}&sortBy=publishedAt&language=en&pageSize={max_results}&apiKey={API_KEY}'
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error al obtener noticias para {company_name}. Código de estado: {response.status_code}")
        print(f"Respuesta: {response.json()}")
        return []
    
    data = response.json()
    articles = data.get('articles', [])
    
    if not articles:
        print(f"No se encontraron noticias para {company_name}. URL de búsqueda: {url}")
    else:
        print(f"Se encontraron {len(articles)} noticias para {company_name}")
    
    return articles

def analyze_sentiment(text):
    return sia.polarity_scores(text)['compound']

def analyze_company_sentiment(company_name, ticker):
    news = get_news(company_name)
    sentiment_data = []
    
    for article in news:
        title = article.get('title', '')
        description = article.get('description', '')
        content = article.get('content', '')
        date = article.get('publishedAt', '')[:10]
        
        full_text = f"{title} {description} {content}"
        sentiment = analyze_sentiment(full_text)
        
        # Solo incluir noticias con sentimiento negativo
        if sentiment < 0:
            sentiment_data.append({
                'Date': date,
                'Company': company_name,
                'Ticker': ticker,
                'Sentiment': sentiment,
                'Title': title
            })
    
    return pd.DataFrame(sentiment_data)

def main():
    all_sentiment_data = pd.DataFrame()

    for company_name, ticker in energy_companies_dict.items():
        company_sentiment = analyze_company_sentiment(company_name, ticker)
        if not company_sentiment.empty:
            all_sentiment_data = pd.concat([all_sentiment_data, company_sentiment], ignore_index=True)
            print(f"Análisis completado para {company_name}. Noticias negativas encontradas: {len(company_sentiment)}")
        else:
            print(f"No se encontraron noticias negativas para {company_name}")

    if all_sentiment_data.empty:
        print("No se encontraron noticias negativas para ninguna empresa.")
    else:
        all_sentiment_data['Date'] = pd.to_datetime(all_sentiment_data['Date'])
        all_sentiment_data = all_sentiment_data.sort_values('Date')

        daily_sentiment = all_sentiment_data.groupby(['Date', 'Company', 'Ticker']).agg({
            'Sentiment': ['mean', 'min', 'max', 'count'],
            'Title': lambda x: x.iloc[0]  # Toma el primer título del día
        }).reset_index()

        daily_sentiment.columns = ['Date', 'Company', 'Ticker', 'Avg_Sentiment', 'Min_Sentiment', 'Max_Sentiment', 'News_Count', 'Sample_Title']

        # Crear el directorio si no existe
        if not os.path.exists(SENTIMENT_PATH):
            os.makedirs(SENTIMENT_PATH)

        # Guardando en formato CSV
        all_sentiment_data.to_csv(os.path.join(SENTIMENT_PATH, 'all_negative_news_sentiment_latam_energy.csv'), index=False)
        daily_sentiment.to_csv(os.path.join(SENTIMENT_PATH, 'daily_negative_sentiment_summary_latam_energy.csv'), index=False)

        print(f"Análisis de sentimiento negativo completado. Los resultados se han guardado en:")
        print(f"- {os.path.join(SENTIMENT_PATH, 'all_negative_news_sentiment_latam_energy.csv')}")
        print(f"- {os.path.join(SENTIMENT_PATH, 'daily_negative_sentiment_summary_latam_energy.csv')}")
        print(f"Total de noticias negativas analizadas: {len(all_sentiment_data)}")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import os

# Ignorar el warning 
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Definir rutas relativas
BASE_PATH = os.path.dirname(os.getcwd())  # Subir un nivel desde la carpeta actual
SENTIMENT_PATH = os.path.join(BASE_PATH, 'datasets', 'dataset_sentimiento')
WEBSCRAPING_PATH = os.path.join(BASE_PATH, 'datasets', 'dataset_webscraping')

# Imprimir las rutas para verificar
print(f"BASE_PATH: {BASE_PATH}")
print(f"SENTIMENT_PATH: {SENTIMENT_PATH}")
print(f"WEBSCRAPING_PATH: {WEBSCRAPING_PATH}")

# Verificar si los archivos existen
sentiment_file = os.path.join(SENTIMENT_PATH, 'daily_negative_sentiment_summary_latam_energy.csv')
stock_file = os.path.join(WEBSCRAPING_PATH, 'latam_energy_stock_data.csv')

if not os.path.exists(sentiment_file):
    print(f"El archivo de sentimiento no existe: {sentiment_file}")
if not os.path.exists(stock_file):
    print(f"El archivo de datos de acciones no existe: {stock_file}")

# Cargar los datos
try:
    sentiment_data = pd.read_csv(sentiment_file)
    stock_data = pd.read_csv(stock_file)
except FileNotFoundError as e:
    print(f"Error al cargar los archivos: {e}")
    raise

# Convertir las fechas a datetime
sentiment_data['Date'] = pd.to_datetime(sentiment_data['Date']).dt.strftime('%Y-%m-%d')
stock_data['Date'] = pd.to_datetime(stock_data['Date'], utc=True).dt.strftime('%Y-%m-%d')

# Datos ordenados por fecha
sentiment_data = sentiment_data.sort_values('Date')
stock_data = stock_data.sort_values('Date')

# Función para calcular el cambio porcentual diario
def calculate_daily_change(group):
    group['Daily_Change'] = group['Close'].pct_change() * 100
    return group

# Calcular el cambio porcentual diario para cada empresa
stock_data = stock_data.groupby('Company', group_keys=False).apply(calculate_daily_change)

# Función para graficar la comparación para una empresa
def plot_sentiment_vs_stock(company):
    company_sentiment = sentiment_data[sentiment_data['Company'] == company]
    company_stock = stock_data[stock_data['Company'] == company]
    
    # Combinar los datos de sentimiento y acciones
    merged_data = pd.merge(company_sentiment, company_stock, on=['Date', 'Company'], how='inner')
    
    if merged_data.empty:
        print(f"No hay datos coincidentes para {company}")
        return
    
    # Convertir la fecha de nuevo a datetime para la gráfica
    merged_data['Date'] = pd.to_datetime(merged_data['Date'])
    
    # Crear la gráfica
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    # Graficar el sentimiento negativo
    ax1.plot(merged_data['Date'], merged_data['Avg_Sentiment'], color='blue', label='Sentimiento Negativo Promedio')
    ax1.set_xlabel('Fecha')
    ax1.set_ylabel('Sentimiento Negativo Promedio', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    
    # Crear un segundo eje y para el cambio porcentual de las acciones
    ax2 = ax1.twinx()
    ax2.plot(merged_data['Date'], merged_data['Daily_Change'], color='red', label='Cambio % Diario')
    ax2.set_ylabel('Cambio % Diario', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    
    # Añadir título y leyenda
    plt.title(f'Sentimiento Negativo vs Cambio % Diario de Acciones - {company}')
    fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)
    
    # Ajustar el diseño
    plt.tight_layout()
    
    # Guardar la gráfica
    plt.savefig(os.path.join(SENTIMENT_PATH, f'{company}_negative_sentiment_vs_stock.png'))
    
    # Mostrar la gráfica
    plt.show()

# Graficar para cada empresa
for company in sentiment_data['Company'].unique():
    plot_sentiment_vs_stock(company)

# Calcular la correlación entre sentimiento negativo y cambio de acciones para cada empresa
correlations = []
for company in sentiment_data['Company'].unique():
    company_sentiment = sentiment_data[sentiment_data['Company'] == company]
    company_stock = stock_data[stock_data['Company'] == company]
    
    merged_data = pd.merge(company_sentiment, company_stock, on=['Date', 'Company'], how='inner')
    
    if not merged_data.empty:
        correlation = merged_data['Avg_Sentiment'].corr(merged_data['Daily_Change'])
        correlations.append({'Company': company, 'Correlation': correlation})
    else:
        print(f"No hay datos coincidentes para calcular la correlación de {company}")

correlation_df = pd.DataFrame(correlations)
correlation_df = correlation_df.sort_values('Correlation', ascending=False)

print("Correlaciones entre Sentimiento Negativo y Cambio % Diario de Acciones:")
print(correlation_df)

# Guardar las correlaciones en un archivo CSV
correlation_df.to_csv(os.path.join(SENTIMENT_PATH, 'negative_sentiment_stock_correlations_latam_energy.csv'), index=False)
print(f"Las correlaciones se han guardado en {os.path.join(SENTIMENT_PATH, 'negative_sentiment_stock_correlations_latam_energy.csv')}")

# Crear un gráfico de barras para las correlaciones
plt.figure(figsize=(12, 6))
plt.bar(correlation_df['Company'], correlation_df['Correlation'])
plt.title('Correlación entre Sentimiento Negativo y Cambio % Diario de Acciones por Empresa de Energía')
plt.xlabel('Empresa')
plt.ylabel('Correlación')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join(SENTIMENT_PATH, 'negative_correlations_bar_chart_latam_energy.png'))
plt.show()